# Librerias

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

# Conexion 

In [2]:
user = "root"
password = "Levp13aa"
host = "localhost"
database = "futbol_dw"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# Limpieza de las columnas 

## Jugadores 

In [3]:
def clean_column_name(col):
    col = str(col).strip()
    col = re.sub(r"[^\w]+", "_", col)
    col = re.sub(r"__+", "_", col)
    col = col.strip("_")
    return col[:60]

In [4]:
player_map_original_to_sql = {
    "Rk": "player_rank",
    "Date": "match_date",
    "Comp": "competition",
    "Age": "player_age",
    "Nation": "nation",
    "Start": "is_starter",
    "Min": "minutes_played",
    "Gls": "goals",
    "Ast": "assists",
    "G+A": "goals_plus_assists",
    "G-PK": "goals_non_penalty",
    "PK": "penalty_goals",
    "PKatt": "penalty_attempts",
    "PKm": "penalty_missed",
    "Sh": "shots_total",
    "SoT": "shots_on_target",
    "SoT%": "shots_on_target_pct",
    "Dist": "shot_avg_distance",
    "FK": "shots_free_kicks",
    "G/Sh": "goals_per_shot",
    "G/SoT": "goals_per_shot_on_target",
    "Att": "passes_attempted",
    "Cmp": "passes_completed",
    "Cmp%": "passes_completion_pct",
    "KP": "key_passes",
    "1/3": "passes_final_third",
    "PPA": "passes_into_penalty_area",
    "CrsPA": "crosses_into_penalty_area",
    "PrgP": "progressive_passes",
    "TotDist": "passes_total_distance",
    "PrgDist": "passes_progressive_distance",
    "PassLive": "passes_live_shot",
    "PassDead": "passes_dead_shot",
    "TO": "take_ons_shot",
    "Sh (shot-creating)": "shots_lead_shot",
    "Fld": "fouls_drawn_shot",  # OJO: en Bronze Fld se duplica, aquí se queda la primera
    "Def": "defensive_actions_shot",
    "PassLive (goal)": "passes_live_goal",
    "PassDead (goal)": "passes_dead_goal",
    "TO (goal)": "take_ons_goal",
    "Sh (goal)": "shots_lead_goal",
    "Fld (goal)": "fouls_drawn_goal",
    "Def (goal)": "defensive_actions_goal",
    "Touches": "touches_total",
    "Def Pen": "touches_def_pen_area",
    "Def 3rd": "touches_def_third",
    "Mid 3rd": "touches_mid_third",
    "Att 3rd": "touches_att_third",
    "Att Pen": "touches_att_pen_area",
    "Live": "touches_live",
    "Tkl": "tackles_total",
    "TklW": "tackles_won",
    "Def 3rd (tkl)": "tackles_def_third",
    "Mid 3rd (tkl)": "tackles_mid_third",
    "Att 3rd (tkl)": "tackles_att_third",
    "Att (vs dribblers)": "dribblers_challenged",
    "Tkl%": "dribblers_tackled_pct",
    "Lost": "dribblers_lost",
    "Blocks": "blocks_total",
    "Sh (block)": "blocks_shot",
    "Pass (block)": "blocks_pass",
    "Int": "interceptions",
    "Tkl+Int": "tackles_plus_interceptions",
    "Clr": "clearances",
    "Err": "errors_lead_shot",
    "CrdY": "yellow_cards",
    "2CrdY": "second_yellow_cards",
    "CrdR": "red_cards",
    "Fls": "fouls_committed",
    # el segundo Fld (fouls_drawn) se pierde en Bronze por duplicado
}
player_clean_map = {clean_column_name(k): v for k, v in player_map_original_to_sql.items()}

Renombrar Bronze_players

In [5]:
dfp = pd.read_sql("SELECT * FROM bronze_players;", engine)

# Renombrar columnas técnicas a nombres futboleros
dfp = dfp.rename(columns=player_clean_map)

# Normalizar nombre de jugador / equipo para futuros joins
if "player_name" in dfp.columns:
    dfp["player_name_clean"] = dfp["player_name"].astype(str).str.lower().str.strip()
elif "Player" in dfp.columns:
    dfp["player_name_clean"] = dfp["Player"].astype(str).str.lower().str.strip()

dfp["team_name_clean"] = dfp["team_name"].astype(str).str.lower().str.strip()

# Asegurarnos de que minutes_played sea numérico
if "minutes_played" in dfp.columns:
    dfp["minutes_played"] = pd.to_numeric(dfp["minutes_played"], errors="coerce")


Hacer metricas por 90 minutos de los jugadores 

In [6]:
per90_base_cols = [
    "goals",
    "assists",
    "goals_plus_assists",
    "shots_total",
    "shots_on_target",
    "key_passes",
    "passes_attempted",
    "passes_completed",
    "progressive_passes",
    "touches_total",
    "tackles_total",
    "interceptions",
    "blocks_total",
    "clearances",
    "yellow_cards",
    "red_cards",
    "fouls_committed",
]

for col in per90_base_cols:
    if col in dfp.columns and "minutes_played" in dfp.columns:
        new_col = f"{col}_per_90"
        dfp[new_col] = np.where(
            dfp["minutes_played"] > 0,
            dfp[col] * 90 / dfp["minutes_played"],
            np.nan
        )

Se normaliza por contexto, es decir, por posicion y competicion 

In [7]:
# Ajusta este nombre si en tu tabla la posición se llama distinto
pos_col = None
for cand in ["position", "Performance_Pos.", "pos"]:
    if cand in dfp.columns:
        pos_col = cand
        break

# Lista de columnas que queremos normalizar (usamos las per_90 y algunas ratios)
norm_cols = [c for c in dfp.columns if c.endswith("_per_90")]
extra_norm_cols = [
    "goals_per_shot",        # goals_per_shot
    "goals_per_shot_on_target",
    "shots_on_target_pct",
    "passes_completion_pct",
]

for c in extra_norm_cols:
    if c in dfp.columns:
        norm_cols.append(c)

def add_zscore(df, group_cols, cols, suffix):
    df = df.copy()
    for c in cols:
        if c not in df.columns:
            continue
        group = df.groupby(group_cols)[c]
        mean = group.transform("mean")
        std = group.transform("std").replace(0, np.nan)
        df[f"{c}{suffix}"] = (df[c] - mean) / std
    return df

# z-score por competición (nivel de liga)
if "competition" in dfp.columns and "season" in dfp.columns:
    dfp = add_zscore(dfp, ["competition", "season"], norm_cols, "_z_comp")

# z-score por posición dentro de competición (si tenemos posición)
if pos_col is not None and "competition" in dfp.columns and "season" in dfp.columns:
    dfp = add_zscore(dfp, ["competition", "season", pos_col], norm_cols, "_z_pos")


 Indices compuestos del jugador

In [ ]:
def safe_mean(df, cols):
    cols = [c for c in cols if c in df.columns]
    if not cols:
        return np.nan
    return df[cols].mean(axis=1)

#  Finishing
dfp["idx_finishing"] = safe_mean(dfp, [
    "goals_per_90_z_pos",
    "goals_per_shot_z_pos",
    "goals_per_shot_on_target_z_pos",
    "shots_on_target_pct_z_pos",
])

#  Generación de ocasiones (playmaking)
dfp["idx_playmaking"] = safe_mean(dfp, [
    "key_passes_per_90_z_pos",
    "passes_into_penalty_area_per_90_z_pos",
    "crosses_into_penalty_area_per_90_z_pos",
    "progressive_passes_per_90_z_pos",
])

#  Progresión de balón
dfp["idx_progression"] = safe_mean(dfp, [
    "progressive_passes_per_90_z_pos",
    "passes_final_third_per_90_z_pos",
    "touches_att_third_per_90_z_pos",
    "touches_att_pen_area_per_90_z_pos",
])

#  Participación en juego
dfp["idx_involvement"] = safe_mean(dfp, [
    "touches_total_per_90_z_pos",
    "passes_attempted_per_90_z_pos",
    "passes_completed_per_90_z_pos",
])

#  Defensa / presión
dfp["idx_defending"] = safe_mean(dfp, [
    "tackles_total_per_90_z_pos",
    "interceptions_per_90_z_pos",
    "blocks_total_per_90_z_pos",
    "clearances_per_90_z_pos",
])

#  Disciplina (tarjetas y faltas)
if "yellow_cards_per_90_z_pos" in dfp.columns:
    dfp["idx_discipline"] = -safe_mean(dfp, [
        "yellow_cards_per_90_z_pos",
        "red_cards_per_90_z_pos",
        "fouls_committed_per_90_z_pos",
    ])


## Equipos 

In [ ]:
team_map_original_to_sql = {
    "Rk": "team_rank",
    "Date": "match_date",
    "Comp": "competition",
    "Round": "match_round",
    "Venue": "venue",
    "Result": "result",
    "GF": "goals_for",
    "GA": "goals_against",
    "Poss": "possession_pct",
    "Sh": "shots_total",
    "SoT": "shots_on_target",
    "SoT%": "shots_on_target_pct",
    "Dist": "shot_avg_distance",
    "FK": "shots_free_kicks",
    "PK": "penalties_scored",
    "PKatt": "penalties_attempted",
    "G/Sh": "goals_per_shot",
    "G/SoT": "goals_per_shot_on_target",
    "Sh/90": "shots_per_90",
    "SoT/90": "shots_on_target_per_90",
    "G/90": "goals_per_90",
    "Ast/90": "assists_per_90",
    "G+A/90": "goals_plus_assists_per_90",
    "xG": "expected_goals",
    "npxG": "non_penalty_xg",
    "xAG": "expected_assisted_goals",
    "npxG+xAG": "non_penalty_xg_plus_xag",
    "PrgC": "progressive_carries",
    "PrgP": "progressive_passes",
    "PrgR": "progressive_receptions",
    "Touches": "touches_total",
    "Def Pen": "touches_def_pen_area",
    "Def 3rd": "touches_def_third",
    "Mid 3rd": "touches_mid_third",
    "Att 3rd": "touches_att_third",
    "Att Pen": "touches_att_pen_area",
    "Live": "touches_live",
    "Passes Att": "passes_attempted",
    "Passes Cmp": "passes_completed",
    "Passes Cmp%": "passes_completion_pct",
    "KP": "key_passes",
    "1/3": "passes_final_third",
    "PPA": "passes_into_penalty_area",
    "CrsPA": "crosses_into_penalty_area",
    "TotDist": "passes_total_distance",
    "PrgDist": "passes_progressive_distance",
    "Tkl": "tackles_total",
    "TklW": "tackles_won",
    "Blocks": "blocks_total",
    "Sh (block)": "blocks_shot",
    "Pass (block)": "blocks_pass",
    "Int": "interceptions",
    "Clr": "clearances",
    "Err": "errors_lead_shot",
    "CrdY": "yellow_cards",
    "CrdR": "red_cards",
    "Fls": "fouls_committed",
    "Fld": "fouls_drawn",
}

# Necesitamos mapear usando los nombres ya "limpios" que hay en bronze_teams
team_clean_map = {clean_column_name(k): v for k, v in team_map_original_to_sql.items()}

# =======================
# 3. Leer Bronze_teams
# =======================

dft = pd.read_sql("SELECT * FROM bronze_teams;", engine)

print(f"bronze_teams -> {dft.shape[0]} filas, {dft.shape[1]} columnas")

# =======================
# 4. Renombrar columnas técnicas a nombres futboleros
# =======================

dft = dft.rename(columns=team_clean_map)

# Asegurarnos de tener team_name, league, season (vienes de Bronze)
# dft["team_name"], dft["league"], dft["season"] ya deberían existir por tu script de carga

# =======================
# 5. Arreglar columnas de porcentaje y tipos numéricos
# =======================

percent_cols = [
    "possession_pct",
    "shots_on_target_pct",
    "passes_completion_pct",
]

for col in percent_cols:
    if col in dft.columns:
        dft[col] = (
            dft[col]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.replace(",", ".", regex=False)
        )
        dft[col] = pd.to_numeric(dft[col], errors="coerce")

# Asegurar numéricos en las métricas principales (por si vienen como string)
numeric_cols = [
    "goals_for", "goals_against",
    "shots_total", "shots_on_target",
    "shot_avg_distance",
    "shots_free_kicks",
    "penalties_scored", "penalties_attempted",
    "goals_per_shot", "goals_per_shot_on_target",
    "shots_per_90", "shots_on_target_per_90",
    "goals_per_90", "assists_per_90", "goals_plus_assists_per_90",
    "expected_goals", "non_penalty_xg", "expected_assisted_goals",
    "non_penalty_xg_plus_xag",
    "progressive_carries", "progressive_passes", "progressive_receptions",
    "touches_total", "touches_def_pen_area", "touches_def_third",
    "touches_mid_third", "touches_att_third", "touches_att_pen_area",
    "touches_live",
    "passes_attempted", "passes_completed",
    "passes_total_distance", "passes_progressive_distance",
    "key_passes", "passes_final_third", "passes_into_penalty_area",
    "crosses_into_penalty_area",
    "tackles_total", "tackles_won",
    "blocks_total", "blocks_shot", "blocks_pass",
    "interceptions", "clearances",
    "errors_lead_shot",
    "yellow_cards", "red_cards",
    "fouls_committed", "fouls_drawn",
]

for col in numeric_cols:
    if col in dft.columns:
        dft[col] = pd.to_numeric(dft[col], errors="coerce")

# =======================
# 6. Calcular z-scores por competición + temporada
# =======================

team_norm_cols = []

# Normalizamos casi todo lo relevante que tengamos
for col in numeric_cols + percent_cols:
    if col in dft.columns:
        team_norm_cols.append(col)

# Evitar duplicados
team_norm_cols = list(sorted(set(team_norm_cols)))

if "competition" in dft.columns and "season" in dft.columns:
    dft = add_zscore(dft, ["competition", "season"], team_norm_cols, "_z_comp")
else:
    print("⚠️ No se encontraron columnas competition y season; no se agregan z-scores.")

# =======================
# 7. Construir índices de estilo de equipo
# =======================

# --- A) Estilo de posesión / juego elaborado ---
dft["idx_possession_style"] = safe_mean(
    dft,
    [
        "possession_pct_z_comp",
        "passes_completed_z_comp",
        "passes_completion_pct_z_comp",
        "touches_total_z_comp",
        "progressive_passes_z_comp",
        "passes_total_distance_z_comp",
    ],
)

# --- B) Estilo ofensivo (capacidad de generar y convertir ocasiones) ---
dft["idx_offensive_style"] = safe_mean(
    dft,
    [
        "goals_for_z_comp",
        "shots_total_z_comp",
        "shots_on_target_z_comp",
        "shots_per_90_z_comp",
        "shots_on_target_per_90_z_comp",
        "expected_goals_z_comp",
        "expected_assisted_goals_z_comp",
        "non_penalty_xg_plus_xag_z_comp",
        "key_passes_z_comp",
        "passes_into_penalty_area_z_comp",
        "crosses_into_penalty_area_z_comp",
        "goals_per_shot_z_comp",
        "goals_per_shot_on_target_z_comp",
        "shots_on_target_pct_z_comp",
    ],
)

# --- C) Estilo defensivo (solidez, capacidad de cortar juego rival) ---
dft["idx_defensive_style"] = safe_mean(
    dft,
    [
        # Menos goles en contra es mejor => lo multiplicamos por -1
        "-goals_against_z_comp" if "goals_against_z_comp" in dft.columns else None,
        "tackles_total_z_comp",
        "blocks_total_z_comp",
        "interceptions_z_comp",
        "clearances_z_comp",
    ],
)

# Convertir la columna de string "-goals_against_z_comp" en valor numérico si existe
if "-goals_against_z_comp" in dft.columns:
    dft["-goals_against_z_comp"] = -dft["goals_against_z_comp"]

# Recalcular idx_defensive_style correctamente ahora que el negativo existe
dft["idx_defensive_style"] = safe_mean(
    dft,
    [
        "-goals_against_z_comp",
        "tackles_total_z_comp",
        "blocks_total_z_comp",
        "interceptions_z_comp",
        "clearances_z_comp",
    ],
)

# --- D) Verticalidad / juego directo / progresión ---
dft["idx_verticality_style"] = safe_mean(
    dft,
    [
        "progressive_passes_z_comp",
        "progressive_carries_z_comp",
        "progressive_receptions_z_comp",
        "passes_final_third_z_comp",
        "passes_into_penalty_area_z_comp",
        "crosses_into_penalty_area_z_comp",
        "shot_avg_distance_z_comp",  # más lejos => más tiros de fuera / más directo
    ],
)

# --- E) Agresividad / intensidad ---
dft["idx_aggressiveness_style"] = safe_mean(
    dft,
    [
        "tackles_total_z_comp",
        "fouls_committed_z_comp",
        "yellow_cards_z_comp",
        "red_cards_z_comp",
    ],
)

# =======================
# 8. Guardar en MySQL como silver_teams
# =======================

dft.to_sql(
    "silver_teams",
    engine,
    if_exists="replace",
    index=False,
    method="multi",
    chunksize=1000,
)

print(f" Tabla 'silver_teams' creada con {dft.shape[0]} filas y {dft.shape[1]} columnas.")